In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

function = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            },
            
        },
        "required": ["questions", "nums"],
    },
}

llm = ChatOpenAI(
    temperature=0.1,
).bind(
    function_call={
        "name": "create_quiz",
    },
    functions=[
        function,
    ],
)

prompt = PromptTemplate.from_template("Make a quiz about {city}")

chain = prompt | llm

response = chain.invoke({"city": "rome"})

response = response.additional_kwargs["function_call"]["arguments"]

response

'{"questions":[{"question":"What year was Rome founded?","answers":[{"answer":"753 BC","correct":true},{"answer":"476 AD","correct":false},{"answer":"1000 AD","correct":false}]},{"question":"Who was the first emperor of Rome?","answers":[{"answer":"Julius Caesar","correct":false},{"answer":"Augustus","correct":true},{"answer":"Nero","correct":false}]},{"question":"What famous structure in Rome was built by Emperor Vespasian?","answers":[{"answer":"Colosseum","correct":true},{"answer":"Pantheon","correct":false},{"answer":"Roman Forum","correct":false}]},{"question":"Which volcano erupted and buried the city of Pompeii near Rome?","answers":[{"answer":"Mount Vesuvius","correct":true},{"answer":"Mount Etna","correct":false},{"answer":"Mount Olympus","correct":false}]}]}'

In [ ]:
import json

for question in json.loads(response)["questions"]:
    print(question)

In [ ]:
import time, os, json
import streamlit as st
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.retrievers import WikipediaRetriever
from langchain.schema import BaseOutputParser

st.set_page_config(
    page_title="QuizGPT",
    page_icon="❓",
    layout='wide',
)

st.title("Quiz GPT")

# -----------------------------------------------------------------------------------------------------------------------------

# def take_open_api_key():
#     if "OPENAI_API_KEY" not in st.session_state:
#         st.session_state['OPENAI_API_KEY'] = st.session_state.open_ai_key
#     else:
#         st.info(f"🟢 API 키 입력 완료 !!")

DIR_NAME = "MyGPT"
BASE_DIR = os.getcwd() if DIR_NAME in os.getcwd() else os.path.join(os.getcwd(), DIR_NAME)
QUIZ_DIR = os.path.join(BASE_DIR, 'uploads/quiz_files')

class JsonOutputParser(BaseOutputParser):
    def parse(self, text):
        text = text.replace("```json", "").replace("json", "")
        return json.loads(text)


llm = ChatOpenAI(
    temperature=0.1,
    model='gpt-4o-mini',
)

questions_prompt = ChatPromptTemplate.from_messages([
    ("system", """
        You are a helpful assistant that is role playing as a teacher.
            
        Based ONLY on the following context make 10 questions to test the user's knowledge about the text.
        
        Each question should have 4 answers, three of them must be incorrect and one should be correct.
            
        Use (o) to signal the correct answer.
            
        Question examples:
            
        Question: What is the color of the ocean?
        Answers: Red | Yellow | Green | Blue(o)
            
        Question: What is the capital or Georgia?
        Answers: Baku | Tbilisi(o) | Manila | Beirut
            
        Question: When was Avatar released?
        Answers: 2007 | 2001 | 2009(o) | 1998
            
        Question: Who was Julius Caesar?
        Answers: A Roman Emperor(o) | Painter | Actor | Model
            
        Your turn!
            
        Context: {context}
    """)
])

formatting_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
    You are a powerful formatting algorithm.
     
    You format exam questions into JSON format.
    Answers with (o) are the correct ones.
     
    Example Input:
    Question: What is the color of the ocean?
    Answers: Red|Yellow|Green|Blue(o)
         
    Question: What is the capital or Georgia?
    Answers: Baku|Tbilisi(o)|Manila|Beirut
         
    Question: When was Avatar released?
    Answers: 2007|2001|2009(o)|1998
         
    Question: Who was Julius Caesar?
    Answers: A Roman Emperor(o)|Painter|Actor|Model
    
     
    Example Output:
     
    ```json
    {{ "questions": [
            {{
                "question": "What is the color of the ocean?",
                "answers": [
                        {{
                            "answer": "Red",
                            "correct": false
                        }},
                        {{
                            "answer": "Yellow",
                            "correct": false
                        }},
                        {{
                            "answer": "Green",
                            "correct": false
                        }},
                        {{
                            "answer": "Blue",
                            "correct": true
                        }},
                ]
            }},
                        {{
                "question": "What is the capital or Georgia?",
                "answers": [
                        {{
                            "answer": "Baku",
                            "correct": false
                        }},
                        {{
                            "answer": "Tbilisi",
                            "correct": true
                        }},
                        {{
                            "answer": "Manila",
                            "correct": false
                        }},
                        {{
                            "answer": "Beirut",
                            "correct": false
                        }},
                ]
            }},
                        {{
                "question": "When was Avatar released?",
                "answers": [
                        {{
                            "answer": "2007",
                            "correct": false
                        }},
                        {{
                            "answer": "2001",
                            "correct": false
                        }},
                        {{
                            "answer": "2009",
                            "correct": true
                        }},
                        {{
                            "answer": "1998",
                            "correct": false
                        }},
                ]
            }},
            {{
                "question": "Who was Julius Caesar?",
                "answers": [
                        {{
                            "answer": "A Roman Emperor",
                            "correct": true
                        }},
                        {{
                            "answer": "Painter",
                            "correct": false
                        }},
                        {{
                            "answer": "Actor",
                            "correct": false
                        }},
                        {{
                            "answer": "Model",
                            "correct": false
                        }},
                ]
            }}
        ]
     }}
    ```
    Your turn!
    Questions: {context}
""",
        )
    ]
)

@st.cache_resource(show_spinner="Loading File...")
def split_file(
        file, 
        file_dir=QUIZ_DIR, 
    ):
    file_content = file.read()
    file_path = os.path.join(file_dir, file.name)
    with open(file_path, "wb") as f:
        f.write(file_content)

    splitter = CharacterTextSplitter.from_tiktoken_encoder(
        separator="\n",
        chunk_size=600,
        chunk_overlap=200,
    )

    loader = UnstructuredFileLoader(file_path)
    docs = loader.load_and_split(text_splitter=splitter)

    return docs

@st.cache_resource(show_spinner="Making quiz . . .")
def run_quiz_chain(docs):
    output_parser = JsonOutputParser()
    chain = {'context': questions_chain} | formatting_chain | output_parser
    return chain.invoke(docs)

@st.cache_resource(show_spinner="Making quiz . . .")
def wiki_search(term):
    retriever = WikipediaRetriever(top_k_results=5, lang='kr')
    with st.status("Searching Wikipedia . . ."):
        docs = retriever.get_relevant_documents(term)

    return docs

def format_docs(docs):
    docs = [document.page_content for document in docs]
    return "\n\n".join(docs)


with st.sidebar:
    docs = None
    choice = st.selectbox("사용방식을 설정하세요.", ("File", "Wikipedia Article"))

    if choice == 'File':
        file = st.file_uploader(".docx, .txt, .pdf 파일을 업로드해주세요.", type=['pdf', 'txt', 'docx'])

        if file:
            docs = split_file(file)
    else:
        topic = st.text_input("Search Wikipedia . . .")
        
        if topic:
            docs = wiki_search(topic)

if not docs:
    st.markdown("""
        환영합니다!
                
        당신이 업로드한 위키백과 기사나 파일에서 퀴즈를 만들어 지식을 테스트하고 공부하는 데 도움을 드리겠습니다.

        사이드바에서 파일을 업로드하거나 위키백과에서 검색하는 것으로 시작하세요.

    """)

else:
    questions_chain = {'context': format_docs} | questions_prompt | llm
    formatting_chain = formatting_prompt | llm
    response = run_quiz_chain(docs)

    with st.form('questions_form'):
        for question in response['questions']:
            value = st.radio("Select an option", [answer['answer'] for answer in question['answer']], index=None)

            if {"answer":value, "correct": True} in question['question']:
                st.success("Correct !!")
            elif value is not None:
                st.error("Wrong !!")
        button = st.form_submit_button()